In [1]:
import pandas as pd

In [2]:
!pip install sqlalchemy pymysql


[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from sqlalchemy import create_engine,text
import urllib.parse

username = "root"
password = urllib.parse.quote_plus("147258369@Arun")  # encodes automatically
host = "localhost"
port = 3306
#database = "sales_ren"

connection_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/"
engine = create_engine(connection_url)
with engine.connect() as conn:
    print("✅ Connected to MySQL server!")

✅ Connected to MySQL server!


In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE DATABASE IF NOT EXISTS sales_ren;"))
    print("✅ Database created successfully!")

✅ Database created successfully!


In [5]:
database = "sales_ren"

connection_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_url)

In [7]:
order=pd.read_csv("../data/orders.csv")
order.to_sql("orders",con=engine,index=False,if_exists="replace")

30000

In [8]:
sales_persons=pd.read_csv("../data/salespersons.csv")
region=pd.read_csv("../data/regions.csv")
products=pd.read_csv("../data/products.csv")
#orders=pd.read_csv("orders.csv")
customers=pd.read_csv("../data/customers.csv")

In [9]:
sales_persons.to_sql("sales_persons",con=engine,index=False,if_exists="replace")
region.to_sql("region",con=engine,index=False,if_exists="replace")
customers.to_sql("customers",con=engine,index=False,if_exists="replace")
products.to_sql("products",con=engine,index=False,if_exists="replace")

6

In [10]:
connection=engine.connect()
result = connection.execute(text("SHOW TABLES;"))

print("Tables in the database:")
for row in result:
    print(row[0])

Tables in the database:
customers
orders
products
region
sales_persons


In [11]:
order.head()

,Order_ID,Order_Date,Customer_ID,Product_ID,Salesperson_ID,Region_ID,Units_Sold,Unit_Price,Cost_Price,Revenue,Profit,Region_Name,Country,Year,Order_Year,Product_Name,Profit_Unit,Day,Month_Name
0,10001,2024-03-23,86,6,15,1,11,489,366,5379,1353,North,USA,2024,2024,Printer,123.0,Saturday,March
1,10002,2024-06-20,694,3,5,3,7,474,327,3318,1029,East,INDIA,2024,2024,Tablet,147.0,Thursday,June
2,10003,2024-01-19,229,3,9,1,11,474,176,5214,3278,North,USA,2024,2024,Tablet,298.0,Friday,January
3,10004,2023-05-11,923,2,5,4,1,787,682,787,105,West,GERMANY,2023,2023,Smartphone,105.0,Thursday,May
4,10005,2024-09-20,154,5,1,3,12,943,829,11316,1368,East,INDIA,2024,2024,Headphones,114.0,Friday,September


In [12]:
quary="""select count(Order_ID) from orders where Region_Name="East" """
cou=pd.read_sql(quary,engine)
print(cou)

   count(Order_ID)
0             7476


In [13]:
quer=""" select o.Customer_ID ,c.Customer_Name,sum(Units_Sold), count(Order_ID) as coun from orders o join customers c on o.Customer_ID=c.Customer_ID group by o.Customer_ID,c.Customer_Name order by coun desc limit 10  """
print(pd.read_sql(quer,engine))

   Customer_ID      Customer_Name  sum(Units_Sold)  coun
0          911       Mary Daniels            409.0    47
1          595        Julian Rose            349.0    47
2          184      Aaron Sellers            422.0    46
3          720    Jonathan Watson            371.0    46
4           62        Casey Small            367.0    45
5          481       Mary Marquez            409.0    44
6           93        Alyssa Wood            343.0    44
7          327        Mary Howard            376.0    44
8          873        Wesley Gray            345.0    44
9          848  Christina Sanchez            300.0    44


In [14]:
order.groupby("Product_ID")["Units_Sold"].sum()

Product_ID
1    39581
2    41424
3    40330
4    38999
5    40236
6    40067
Name: Units_Sold, dtype: int64